In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os
from xgboost import XGBClassifier
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [ ]:
file_path = "Processed_03.csv"
data = pd.read_csv(file_path)

X = data.drop(columns=['Attack Type'])
y = data['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

model_path = "./data/best_rf_model.pkl"
joblib.dump(best_rf, model_path)

y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

report_path = "./data/rf_classification_report.txt"
with open(report_path, "w") as f:
    f.write(f"Best Model Accuracy: {accuracy * 100:.2f}%\n\n")
    f.write("Classification Report:\n")
    f.write(classification_rep)

accuracy, classification_rep, model_path, report_path

Fitting 5 folds for each of 81 candidates, totalling 405 fits


(0.33016666666666666,
 '              precision    recall  f1-score   support\n\n           0       0.33      0.30      0.32      3992\n           1       0.33      0.28      0.30      3980\n           2       0.33      0.41      0.37      4028\n\n    accuracy                           0.33     12000\n   macro avg       0.33      0.33      0.33     12000\nweighted avg       0.33      0.33      0.33     12000\n',
 './data/best_rf_model.pkl',
 './data/rf_classification_report.txt')

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0]
}

grid_search = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42), 
                           param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_

save_dir = "./data"
os.makedirs(save_dir, exist_ok=True)

xgb_model_path = os.path.join(save_dir, "best_xgb_model.pkl")
report_path = os.path.join(save_dir, "xgb_classification_report.txt")

joblib.dump(best_xgb, xgb_model_path)

y_pred = best_xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

with open(report_path, "w") as f:
    f.write(f"Best Model Accuracy: {accuracy * 100:.2f}%\n\n")
    f.write("Classification Report:\n")
    f.write(classification_rep)

accuracy, classification_rep, xgb_model_path, report_path

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


(0.33716666666666667,
 '              precision    recall  f1-score   support\n\n           0       0.34      0.33      0.34      3992\n           1       0.33      0.34      0.33      3980\n           2       0.34      0.34      0.34      4028\n\n    accuracy                           0.34     12000\n   macro avg       0.34      0.34      0.34     12000\nweighted avg       0.34      0.34      0.34     12000\n',
 './data\\best_xgb_model.pkl',
 './data\\xgb_classification_report.txt')

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return self.softmax(x)

input_size = X_train.shape[1]
num_classes = len(np.unique(y))
model = NeuralNetwork(input_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate Model
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor)
    y_pred = torch.argmax(y_pred_probs, dim=1).numpy()

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

model_path = os.path.join(save_dir, "best_nn_model.pth")
torch.save(model.state_dict(), model_path)

report_path = os.path.join(save_dir, "nn_classification_report.txt")
with open(report_path, "w") as f:
    f.write(f"Best Model Accuracy: {accuracy * 100:.2f}%\n\n")
    f.write("Classification Report:\n")
    f.write(classification_rep)

accuracy, classification_rep, model_path, report_path
